**COVID 19 Data Exploration**

**Skills used: Joins, CTE's, Aggregate Functions, Creating views, Converting data types.**

In [ ]:
-- looking at top 500 data from the covid_death table
SELECT top (500) location, date, total_cases, new_cases, total_deaths, population
from covid_project.dbo.covid_death
where continent is not null
order by 1,2

In [ ]:
-- showing total cases and death with the percentage death rate
SELECT location, date, total_cases, total_deaths, (total_deaths/total_cases)*100 as deathpercentage
from covid_project.dbo.covid_death
where location like '%nigeria%'
order by deathpercentage DESC

In [ ]:
-- total cases by location to show what percentage of the population is infected with covid 19
SELECT location, date, population,  total_cases,(total_cases/population)*100 as populationpercent
from covid_project.dbo.covid_death
where location = 'nigeria'
order by populationpercent DESC

In [ ]:
-- countries with highest infection rate against population
SELECT location,population, MAX(total_cases) as HighestInfectionCount ,MAX((total_cases/population))*100 as percentinfected
from covid_project.dbo.covid_death
group by location , population
order by percentinfected desc

In [ ]:
-- showng countries with highest death per population
SELECT location, MAX(cast(total_deaths as int)) as TotalDeathCount
from covid_project.dbo.covid_death
where continent is not NUll
group by location 
order by TotalDeathCount desc

In [ ]:
-- showing continent with the highest death count per population
SELECT location, MAX(cast(total_deaths as int)) as TotalDeathCount
from covid_project.dbo.covid_death
where continent is NUll
group by [location]
order by TotalDeathCount desc

In [ ]:
-- GLOBAL EXPLORATION
SELECT location, SUM(NEW_CASES) AS total_cases, SUM(CAST(NEW_DEATHS AS int)) AS total_deaths, SUM(CAST(NEW_DEATHS AS INT))/SUM(NEW_CASES)*100 AS TOTALDEATHPERCENT
FROM covid_project.DBO.Covid_death
WHERE continent IS NOT NULL
GROUP BY [location]
ORDER BY TOTALDEATHPERCENT DESC

In [ ]:
-- Total Population vs Vaccinations
-- Shows Percentage of Population that has recieved at least one Covid Vaccine

Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
From covid_project.dbo.Covid_death dea
Join covid_project.dbo.Covid_Vac
cation = vac.location
	and dea.date = vac.date
where dea.continent is not null 
order by 3

In [ ]:
With PopvsVac (Continent, Location, Date, Population, New_Vaccinations, RollingPeopleVaccinated)
as
(
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
From covid_project.dbo.Covid_death dea
Join covid_project.dbo.Covid_Vacc vac
	On dea.location = vac.location
	and dea.date = vac.date
where dea.continent is not null and dea.population is not null
--order by 2,3
)
Select *, (RollingPeopleVaccinated/Population)*100 as percenagevaccinated
From PopvsVac

In [ ]:
-- Using Temp Table to perform Calculation on Partition By in previous query

DROP Table if exists #PercentPopulationVaccinated
Create Table #PercentPopulationVaccinated
(
Continent nvarchar(255),
Location nvarchar(255),
Date datetime,
Population numeric,
New_vaccinations numeric,
RollingPeopleVaccinated numeric
)

Insert into #PercentPopulationVaccinated
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
From covid_project.dbo.Covid_death dea
Join covid_project.dbo.Covid_Vac

In [ ]:
Select *, (RollingPeopleVaccinated/Population)*100
From #PercentPopulationVaccinated

In [1]:
-- Creating View to store data for later visualizations

Create View PercentPopulationVaccinated as
Select dea.continent, dea.location, dea.date, dea.population, vac.new_vaccinations
, SUM(CONVERT(int,vac.new_vaccinations)) OVER (Partition by dea.Location Order by dea.location, dea.Date) as RollingPeopleVaccinated
--, (RollingPeopleVaccinated/population)*100
From covid_project.dbo.Covid_death dea
Join covid_project.dbo.Covid_Vacc vac
	On dea.location = vac.location
	and dea.date = vac.date
where dea.continent is not null 

Commands completed successfully.

Total execution time: 00:00:02.518